# miTag analysis

## Import data from uclust taxonomy assignment 
### 10/05/2017
### Output from assign taxonomy on rRNA separated reads is input as miTags here.

In [ ]:
library(reshape2)
library(plyr)
library(dplyr)
library(tidyr)
library(ggplot2)
library(gridExtra)

## Run below function to import raw taxonomy assignments from QIIME output.
## Alternatively, run from miTag table already created, see below.

In [ ]:
#All files listed from each QIIME assign taxonomy output.
#miTagFiles<-c("/beleriand/Metatranscriptome_analysis/miTag_dir/Tax_assign_all/SPOT_150m_25_uclust/SPOT_150m_25_R1_tax_assignments.txt","/beleriand/Metatranscriptome_analysis/miTag_dir/Tax_assign_all/SPOT_150m_26_uclust/SPOT_150m_26_R1_tax_assignments.txt","/beleriand/Metatranscriptome_analysis/miTag_dir/Tax_assign_all/SPOT_150m_27_uclust/SPOT_150m_27_R1_tax_assignments.txt","/beleriand/Metatranscriptome_analysis/miTag_dir/Tax_assign_all/SPOT_890m_28_uclust/SPOT_890m_28_R1_tax_assignments.txt","/beleriand/Metatranscriptome_analysis/miTag_dir/Tax_assign_all/SPOT_890m_29_uclust/SPOT_890m_29_R1_tax_assignments.txt","/beleriand/Metatranscriptome_analysis/miTag_dir/Tax_assign_all/SPOT_890m_30_uclust/SPOT_890m_30_R1_tax_assignments.txt","/beleriand/Metatranscriptome_analysis/miTag_dir/Tax_assign_all/SPOT_890m_31_uclust/SPOT_890m_31_R1_tax_assignments.txt","/beleriand/Metatranscriptome_analysis/miTag_dir/Tax_assign_all/SPOT_surface_10_uclust/SPOT_surface_10_R1_tax_assignments.txt","/beleriand/Metatranscriptome_analysis/miTag_dir/Tax_assign_all/SPOT_surface_11_uclust/SPOT_surface_11_R1_tax_assignments.txt","/beleriand/Metatranscriptome_analysis/miTag_dir/Tax_assign_all/SPOT_surface_12_uclust/SPOT_surface_12_R1_tax_assignments.txt","/beleriand/Metatranscriptome_analysis/miTag_dir/Tax_assign_all/SPOT_surface_7_uclust/SPOT_surface_7_R1_tax_assignments.txt","/beleriand/Metatranscriptome_analysis/miTag_dir/Tax_assign_all/SPOT_surface_8_uclust/SPOT_surface_8_R1_tax_assignments.txt","/beleriand/Metatranscriptome_analysis/miTag_dir/Tax_assign_all/SPOT_surface_9_uclust/SPOT_surface_9_R1_tax_assignments.txt")

In [ ]:
#rm(dataset);rm(tmpdata) #Make sure objects do not exist already
#Function to import each file, modify, and concatenate
for (file in miTagFiles){
  if (!exists("dataset")){
    infile<-read.delim(file, header=FALSE)
    infile$Count<-1
    split<-colsplit(infile$V1, "_", c("Location", "Depth", "Rep", "Excess")); split$Excess <-NULL
    infile2<-cbind(split,infile)
    infile2$Sample<-paste(infile2$Location, infile2$Depth, sep="_")
    dataset<-aggregate(infile2$Count, by=list(Sample=infile2$Sample, Taxonomy=infile2$V2),sum)
  }
  if (exists("dataset")){
    infile<-read.delim(file, header=FALSE)
    infile$Count<-1
    split<-colsplit(infile$V1, "_", c("Location", "Depth", "Rep", "Excess")); split$Excess <-NULL
    infile2<-cbind(split,infile)
    infile2$Sample<-paste(infile2$Location, infile2$Depth, sep="_")
    tmpdata<-aggregate(infile2$Count, by=list(Sample=infile2$Sample, Taxonomy=infile2$V2),sum)
    dataset<-rbind(dataset, tmpdata)
    rm(tmpdata)
  }
}
head(dataset)

In [ ]:
### Split Taxonomy column into multiple
#dataset1<-dataset
#Find and replace syntax
#dataset1$Tax<-gsub("D_([0-9]*)__","", dataset1$Taxonomy)
#dataset1$Tax<-gsub(";;","", dataset1$Tax)
#tmp<-colsplit(dataset1$Tax,';',c("Lev1", "Lev2", "Lev3", "Lev4", "Lev5", "Lev6", "Lev7", "Lev8"))
#summed_miTags<-cbind(dataset1[c(1,3)],tmp)
#colnames(summed_miTags)[2]<-"Count"
#save(summed_miTags, file="summed_miTags")

In [ ]:
#Above may need to run overnight. Pick up from here.
#load("summed_miTags")
#write.table(summed_miTags, file="miTags_alldepths.txt", quote=FALSE, sep="\t", row.names=FALSE)
summed_miTags<-read.delim("miTags_alldepths.txt")

In [ ]:
head(summed_miTags)

In [ ]:
#Manual curation of taxonomy list:
summed_miTags$Taxa<-"Unassigned"
summed_miTags$Taxa[summed_miTags$Lev1 == "Archaea"]="Archaea"
summed_miTags$Taxa[summed_miTags$Lev1 == "Bacteria"]="Bacteria"
summed_miTags$Taxa[summed_miTags$Lev1 == "Eukaryota"]="Eukaryota"
domain<-aggregate(summed_miTags$Count, by=list(Domain=summed_miTags$Taxa, Sample=summed_miTags$Sample),sum)
#
#Subset only eukaryotic miTags and update the taxonomy
euks<-subset(summed_miTags, Taxa %in% "Eukaryota") 
euks$Taxa[euks$Lev2 == "Archaeplastida"]="Archaeplastid"
euks$Taxa[euks$Lev2 == "Amoebozoa"]="Other eukaryote"
euks$Taxa[euks$Lev3 == "Chloroplastida"]="Chlorophyte" #explore further at Lev6 and 7
euks$Taxa[euks$Lev3 == "Rhodophyceae"]="Rhodophyte"
euks$Taxa[euks$Lev2 == "Cryptophyceae"]="Cryptophyte"
euks$Taxa[euks$Lev2 == "Excavata"]="Excavates"
euks$Taxa[euks$Lev2 == "Haptophyta"]="Haptophyte" #further at Lev4
euks$Taxa[euks$Lev2 == "Opisthokonta"]="Metazoa"
euks$Taxa[euks$Lev4 == "Fungi"]="Metazoa"
euks$Taxa[euks$Lev2 == "SAR"]="Other eukaryote"
euks$Taxa[euks$Lev4 == "Apicomplexa"]="Other eukaryote" #no other way to classify
euks$Taxa[euks$Lev3 == "Alveolata"]="Other Alveolate"
euks$Taxa[euks$Lev3 == "uncultured ciliate"]="Ciliate" 
euks$Taxa[euks$Lev4 == "Ciliophora"]="Ciliate" #Explore further at Lev7
euks$Taxa[euks$Lev4 == "Dinoflagellata"]="Dinoflagellate" #Explore further at Lev7
euks$Taxa[euks$Lev2 == "uncultured dinoflagellate"]="Dinoflagellate"
euks$Taxa[euks$Lev5 == "Syndiniales"] = "Syndiniales"
euks$Taxa[euks$Lev3 == "Rhizaria"]="Other Rhizaria"
euks$Taxa[euks$Lev4 == "Cercozoa"]="Cercozoa" #lev5
euks$Taxa[euks$Lev5 == "Retaria"]="Retaria" #lev5
euks$Taxa[euks$Lev3 == "Stramenopiles"]="Other Stramenopile" #lev4 for more
mast<-c("MAST-1","MAST-11","MAST-12","MAST-16","MAST-2","MAST-22","MAST-23","MAST-24","MAST-3","MAST-4","MAST-6","MAST-7","MAST-8","MAST-9")
euks$Taxa[euks$Lev4 %in% mast]="MAST"
euks$Taxa[euks$Lev5 == "Pelagophyceae"]="Pelagophyte"
euks$Taxa[euks$Lev5 == "Chrysophyceae"]="Chrysophyte"
euks$Taxa[euks$Lev5 == "Diatomea"]="Diatom" # Lev8 for more!
other<-c("Ambiguous_taxa","AB3F14RM1B12","BW-dinoclone28","Centrohelida","D4P07G08","DH147-EKD10","Incertae Sedis", "SA1-3C06", "uncultured marine eukaryote", "Protosteliales sp. JvW-2015", "", "Picozoa")
euks$Taxa[euks$Lev2 %in% other]="Other eukaryote"
#
euks$Taxa_simple<-euks$Taxa
euks$Taxa_simple[euks$Taxa == "Rhodophyte"]="Archaeplastid"
euks$Taxa_simple[euks$Taxa == "Other Alveolate"]="Other eukaryote"
rhiz<-c("Other Rhizaria", "Cercozoa", "Retaria")
euks$Taxa_simple[euks$Taxa %in% rhiz]="Rhizaria"
euks$Taxa_simple[euks$Taxa == "Other Stramenopile"]="Other eukaryote"
euks$Taxa_simple[euks$Taxa == "Chrysophyte"]="Other eukaryote"
euks$Taxa_simple[euks$Taxa == "Cryptophyte"]="Other eukaryote"


In [ ]:
#For generate Figure 2B
simple_miTag<-aggregate(euks$Count, by=list(Sample=euks$Sample, Taxa_simple=euks$Taxa_simple),sum)
#Generate table of miTag results
euk_summary<-aggregate(euks$Count, by=list(Taxa_simple=euks$Taxa_simple,Taxa=euks$Taxa,Sample=euks$Sample),sum)

In [ ]:
#save(euks,domain,euk_summary,simple_miTag, file="miTag_dfs.RData") #optional checkpoint to save dfs
load("miTag_dfs.RData",verbose=T)

In [ ]:
#Plot domain
domain$var<-factor(domain$Sample, levels=c("SPOT_890m", "SPOT_150m", "SPOT_surface"), labels = c("890 m", "150 m", "Surface"))
domain_color<-c("#b2182b", "#fed976", "#2171b5", "grey")
#
plot_domain<-ggplot(domain,aes(y=x,x=var,fill=Domain))+geom_bar(stat="identity", position="fill", color="#525252")+labs(title="", x="",y="miTag relative abundance")+scale_y_continuous(position = "top", expand=c(0,0))+theme(legend.title=element_blank(),legend.position="right",legend.text.align=0, axis.text = element_text(color="black"),panel.grid.major = element_blank(), panel.grid.minor = element_blank(),panel.background = element_blank(),panel.border = element_blank(), axis.line = element_line())+coord_flip()+scale_x_discrete(limits=c(), expand = c(0, 0))+scale_fill_manual(values=domain_color)

plot_domain

In [ ]:
#Plot taxonomy assignment to miTags
##Figure 2B
tax_order2=c("Dinoflagellate","Syndiniales","Ciliate","Haptophyte","Archaeplastid","Chlorophyte","MAST","Diatom","Pelagophyte","Excavates","Rhizaria","Other eukaryote","Metazoa")
tax_color2<-c("#c2a5cf","#6a51a3","#d53e4f","#f46d43","#006d2c","#5aae61","#c51b8a","#e6f598","#feb24c","#e5f5e0","#a6bddb","#74add1","#737373")
names(tax_color2)<-tax_order2
colScale2<-scale_colour_manual(values=tax_color2)
simple_miTag$tax<-factor(simple_miTag$Taxa_simple, levels=rev(tax_order2))
simple_miTag$var<-factor(simple_miTag$Sample, levels=c("SPOT_890m", "SPOT_150m", "SPOT_surface"), labels = c("890 m", "150 m", "Surface"))

plot_miTag_simple<-ggplot(simple_miTag[order(simple_miTag$tax),],aes(y=x,x=var,fill=tax))+geom_bar(stat="identity", position="fill", color="black")+labs(title="", x="",y="Relative abundance miTag")+scale_x_discrete(limits=c(), expand = c(0, 0))+scale_fill_manual(values=tax_color2)+coord_flip()+scale_y_continuous(position = "top", expand=c(0,0))+theme(legend.title=element_blank(),legend.position="right",legend.text.align=0, axis.text = element_text(color="black"),panel.grid.major = element_blank(), panel.grid.minor = element_blank(),panel.background = element_blank(),panel.border = element_blank(), axis.line = element_line())
#
plot_miTag_simple

In [ ]:
#Generate Table S7 to look at taxonomic breakdown:
miTag_table<-dcast(euk_summary, Taxa_simple+Taxa~Sample, fill=0)
#write.table(miTag_table, file="TableS7.txt", quote=FALSE, sep="\t", row.names=FALSE)
euks_all<-dcast(euks[c(12,11,3:6,1:2)], Taxa_simple+Taxa+Lev1+Lev2+Lev3+Lev4~Sample, fun.aggregate = sum)
#write.table(euks_all, "miTag_fulloutput.txt", quote=FALSE, sep="\t", row.names=FALSE)

In [ ]:
#Look at taxa of interest
#Figure S1 
euks_2<-euks
euks_2$Taxa2<-"none"
euks_2$Lev7<-as.character(euks_2$Lev7)
euks_2$Taxa2<-with(euks_2, ifelse(Taxa %in% "Chlorophyte", Lev7, Taxa2))
euks_2$Lev4<-as.character(euks_2$Lev4)
euks_2$Taxa2<-with(euks_2, ifelse(Taxa %in% "Haptophyte", Lev4, Taxa2))
euks_2$Taxa2<-with(euks_2, ifelse(Taxa %in% "Ciliate", Lev7, Taxa2))
euks_2$Taxa2<-with(euks_2, ifelse(Taxa %in% "Dinoflagellate", Lev7, Taxa2))
euks_2$Lev8<-as.character(euks_2$Lev8)
euks_2$Taxa2<-with(euks_2, ifelse(Taxa %in% "Diatom", Lev8, Taxa2))
euks_2$Taxa2<-with(euks_2, ifelse(Taxa2 %in% "", paste(euks_2$Taxa, "Uncertain", sep=" "), Taxa2))
euks_2<-as.data.frame(euks_2)
head(euks_2)

In [ ]:
#Remove taxa we aren't interested in
tax<-c("Dinoflagellate","Ciliate","Haptophyte","Diatom","Chlorophyte")
tmp1<-subset(euks_2, Taxa %in% tax)
#Sum again by Taxa 2
euks_3<-aggregate(tmp1$Count, by=list(Sample=tmp1$Sample, Taxa=tmp1$Taxa, Taxa2=tmp1$Taxa2),sum)
head(euks_3)

In [ ]:
#First subset so that all hits are >500
#Then, subset to the top 5 taxonomic groups to look at further resolution
euks_4<-subset(euks_3, x>500)
dim(euks_4)
result <- euks_4 %>% 
  group_by(Sample, Taxa) %>%
  top_n(n=5) %>%
  arrange(Taxa,Taxa2,Sample)           
top5<-as.data.frame(result)


In [ ]:
length(unique(top5$Taxa2))

In [ ]:
unique(top5$Taxa)
unique(top5$Taxa2)

In [ ]:
#Plot each
top5$var<-factor(top5$Sample, levels=c("SPOT_890m", "SPOT_150m", "SPOT_surface"), labels = c("890 m", "150 m", "Surface"))
#
plot_top5<-ggplot(top5,aes(y=x,x=var,fill=Taxa2))+geom_bar(stat="identity", position="fill", color="#525252")+labs(title="", x="",y="Top 5 miTags in each taxonomic group")+scale_x_discrete(limits=c(), expand = c(0, 0))+coord_flip()+scale_y_continuous(position = "top", expand=c(0,0))+theme(legend.title=element_blank(),legend.position="right",legend.text.align=0, axis.text = element_text(color="black"),panel.grid.major = element_blank(), panel.grid.minor = element_blank(),panel.background = element_blank(),panel.border = element_blank(), axis.line = element_line())

In [ ]:
plot_top5 %+% subset(top5, Taxa %in% "Dinoflagellate")+scale_fill_brewer(palette="Purples")

In [ ]:
plot_top5 %+% subset(top5, Taxa %in% "Ciliate")+scale_fill_brewer(palette="Reds")

In [ ]:
plot_top5 %+% subset(top5, Taxa %in% "Haptophyte")+scale_fill_brewer(palette="Oranges")

In [ ]:
plot_top5 %+% subset(top5, Taxa %in% "Diatom")+scale_fill_brewer(palette="YlOrRd")

In [ ]:
plot_top5 %+% subset(top5, Taxa %in% "Chlorophyte")+scale_fill_brewer(palette="Greens")